In [1]:
pip install IMDbPY

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 9.5 MB/s eta 0:00:00


In [2]:
from imdb import IMDb
import platform
import subprocess
import requests
from bs4 import BeautifulSoup

def fetch_movie_description(movie_name):
    ia = IMDb()
    movies = ia.search_movie(movie_name)
    if movies:
        movie = movies[0]
        ia.update(movie)
        return movie.get('plot outline')
    else:
        return None

def copy_to_clipboard_windows(text):
    """Copy text to clipboard on Windows using clip"""
    try:
        subprocess.run(['clip'], input=text.encode('utf-8'), check=True)
        return True
    except Exception as e:
        print(f"Windows clipboard error: {e}")
        return False

def copy_to_clipboard_linux(text):
    """Copy text to clipboard on Linux using xclip or xsel"""
    # Try xclip first
    try:
        subprocess.run(['xclip', '-selection', 'clipboard'], input=text.encode('utf-8'), check=True)
        return True
    except FileNotFoundError:
        # If xclip isn't available, try xsel
        try:
            subprocess.run(['xsel', '--clipboard', '--input'], input=text.encode('utf-8'), check=True)
            return True
        except FileNotFoundError:
            print("Neither xclip nor xsel is installed. Please install one of them:")
            print("sudo apt-get install xclip  # or")
            print("sudo apt-get install xsel")
            return False
    except Exception as e:
        print(f"Linux clipboard error: {e}")
        return False

def copy_to_clipboard_macos(text):
    """Copy text to clipboard on macOS using pbcopy"""
    try:
        subprocess.run(['pbcopy'], input=text.encode('utf-8'), check=True)
        return True
    except Exception as e:
        print(f"macOS clipboard error: {e}")
        return False

def copy_to_clipboard(text):
    """
    Platform-independent clipboard copy function
    """
    system = platform.system().lower()
    success = False

    if system == 'windows':
        success = copy_to_clipboard_windows(text)
    elif system == 'linux':
        success = copy_to_clipboard_linux(text)
    elif system == 'darwin':  # macOS
        success = copy_to_clipboard_macos(text)
    else:
        print(f"Unsupported operating system: {system}")

    if success:
        print("Successfully copied prompt to clipboard!")
    else:
        print("Clipboard operation failed. Here's your text to copy manually:")
        print("-" * 50)
        print(text)
        print("-" * 50)
        # Create a text file as backup
        try:
            with open('generated_prompt.txt', 'w', encoding='utf-8') as f:
                f.write(text)
            print("Text has been saved to 'generated_prompt.txt'")
        except Exception as e:
            print(f"Could not save to file: {e}")

def create_persuasive_prompt(movie_name, persuasion_strategy, physique, uncertainty_avoidance, personality_type):
    movie_description = fetch_movie_description(movie_name)
    if not movie_description:
        return "Movie not found."

    prompt = (
        f"Here is the Movie description of '{movie_name}' from the IMDb website: {movie_description}\n"
        f"Create a persuasive explanation for a person whose dominant persuasion strategy is '{persuasion_strategy}', "
        f"is '{physique}', has '{uncertainty_avoidance}' uncertainty avoidance, and is '{personality_type}, Start your answer with Here is a persuasive explanation of the movie '{movie_name}' for a person with a '{persuasion_strategy}' dominant persuasion strategy, '{physique}' personality, '{uncertainty_avoidance}' uncertainty avoidance, and '{personality_type} ' tendencies . Then show what each part of your response corresponds to each aspect of the reader's personality. "
    )

    # Try to copy to clipboard
    copy_to_clipboard(prompt)
    return prompt

def main():
    print("\n=== Movie Prompt Generator ===")
    try:
        # Get user input with input validation
        movie_name = input("Enter the name of the movie: ").strip()
        while not movie_name:
            print("Movie name cannot be empty!")
            movie_name = input("Enter the name of the movie: ").strip()

        persuasion_strategy = input("Enter the dominant persuasion strategy: ").strip()
        while not persuasion_strategy:
            print("Persuasion strategy cannot be empty!")
            persuasion_strategy = input("Enter the dominant persuasion strategy: ").strip()

        physique = input("Enter 'feminine' or 'muscular': ").strip().lower()
        while physique not in ['feminine', 'muscular']:
            print("Please enter either 'feminine' or 'muscular'!")
            physique = input("Enter 'feminine' or 'muscular': ").strip().lower()

        uncertainty_avoidance = input("Enter 'high' or 'low' uncertainty avoidance: ").strip().lower()
        while uncertainty_avoidance not in ['high', 'low']:
            print("Please enter either 'high' or 'low'!")
            uncertainty_avoidance = input("Enter 'high' or 'low' uncertainty avoidance: ").strip().lower()

        personality_type = input("Enter 'introvert' or 'extrovert': ").strip().lower()
        while personality_type not in ['introvert', 'extrovert']:
            print("Please enter either 'introvert' or 'extrovert'!")
            personality_type = input("Enter 'introvert' or 'extrovert': ").strip().lower()

        # Generate and display prompt
        print("\nGenerating prompt...")
        prompt = create_persuasive_prompt(
            movie_name,
            persuasion_strategy,
            physique,
            uncertainty_avoidance,
            personality_type
        )
        print("\nGenerated Prompt:\n", prompt)

    except KeyboardInterrupt:
        print("\nOperation cancelled by user.")
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")

if __name__ == "__main__":
    main()


=== Movie Prompt Generator ===
Enter the name of the movie: Inside Man
Enter the dominant persuasion strategy: Liking
Enter 'feminine' or 'muscular': muscular
Enter 'high' or 'low' uncertainty avoidance: high
Enter 'introvert' or 'extrovert': extrovert

Generating prompt...
Neither xclip nor xsel is installed. Please install one of them:
sudo apt-get install xclip  # or
sudo apt-get install xsel
Clipboard operation failed. Here's your text to copy manually:
--------------------------------------------------
Here is the Movie description of 'Inside Man' from the IMDb website: Composed and meticulous, the soft-spoken and ingenious bank robber, Dalton Russell, has orchestrated the perfect heist--shortly, the Manhattan bank at 20 Exchange Place along with dozens of hostages will be his to command. Try as he might, the rough hostage negotiator, Keith Frazier, is always one step behind the criminal mastermind--and what is more disheartening--the institution's silver-haired founder, Arthur C

In [3]:
def extract_characteristics_from_text(text):
    """
    Extracts all relevant characteristics from text.
    Returns a dictionary of detected characteristics.
    """
    text = text.lower()
    characteristics = {}

    # Persuasion Strategy
    if 'unity' in text:
        characteristics['persuasion'] = 'Unity'
    elif 'authority' in text:
        characteristics['persuasion'] = 'Authority'
    elif 'liking' in text:
        characteristics['persuasion'] = 'Liking'

    # Uncertainty Avoidance
    if 'high uncertainty' in text or 'risk averse' in text:
        characteristics['uncertainty'] = 'High'
    elif 'low uncertainty' in text or 'risk taking' in text:
        characteristics['uncertainty'] = 'Low'

    # Masculinity/Femininity
    if 'muscular' in text or 'masculinity' in text:
        characteristics['gender_dimension'] = 'Masculine'
    elif 'feminine' in text or 'femininity' in text:
        characteristics['gender_dimension'] = 'Feminine'

    # Personality Type
    if 'introvert' in text or 'introverted' in text:
        characteristics['personality'] = 'Introvert'
    elif 'extrovert' in text or 'extroverted' in text:
        characteristics['personality'] = 'Extrovert'

    return characteristics

def generate_analysis_prompt(text_content):
    """
    Generates a comprehensive analysis prompt based on detected characteristics.
    """
    characteristics_guide = {
        'Unity': (
            "Create an explanation that:\n"
            "- Emphasizes shared experiences and common values\n"
            "- Highlights collective journey and mutual understanding\n"
            "- Shows how the story reflects universal human experiences\n"
            "- Creates a sense of belonging and community connection\n"
            "- Demonstrates how the protagonist's journey relates to common life experiences"
        ),
        'Authority': (
            "Create an explanation that:\n"
            "- Emphasizes expert opinions and critical acclaim\n"
            "- Highlights awards, recognition, and professional reviews\n"
            "- References influential figures who praised the film\n"
            "- Includes statistics and expert analysis\n"
            "- Demonstrates the film's impact on cinema and culture"
        ),
        'Liking': (
            "Create an explanation that:\n"
            "- Emphasizes emotional connections and relatable characters\n"
            "- Highlights personal and touching moments\n"
            "- Shows how the story creates emotional bonds\n"
            "- Focuses on likable aspects of characters\n"
            "- Demonstrates how viewers develop attachment to the story"
        ),
        'High': (
            "Consider these uncertainty avoidance aspects:\n"
            "- Preference for clear rules and structured situations\n"
            "- Focus on planning and risk mitigation\n"
            "- Emphasis on stability and predictability\n"
            "- Attention to detail and careful decision-making\n"
            "- Resistance to unconventional or ambiguous elements"
        ),
        'Low': (
            "Consider these uncertainty acceptance aspects:\n"
            "- Comfort with ambiguous or unstructured situations\n"
            "- Embrace of risk and innovation\n"
            "- Flexibility in approach and methodology\n"
            "- Openness to new experiences and changes\n"
            "- Tolerance for unexpected developments"
        ),
        'Masculine': (
            "Consider these masculine cultural elements:\n"
            "- Focus on achievement and success\n"
            "- Emphasis on competition and assertiveness\n"
            "- Value placed on material rewards\n"
            "- Clear goal-oriented actions\n"
            "- Prominence of ambitious pursuits"
        ),
        'Feminine': (
            "Consider these feminine cultural elements:\n"
            "- Focus on quality of life and relationships\n"
            "- Emphasis on cooperation and consensus\n"
            "- Value placed on caring and nurturing\n"
            "- Attention to work-life balance\n"
            "- Importance of social connections"
        ),
        'Introvert': (
            "Consider these introverted characteristics:\n"
            "- Internal processing and reflection\n"
            "- Preference for deeper, one-on-one interactions\n"
            "- Need for personal space and solitude\n"
            "- Thoughtful and measured responses\n"
            "- Focus on internal growth and understanding"
        ),
        'Extrovert': (
            "Consider these extroverted characteristics:\n"
            "- External processing and social engagement\n"
            "- Preference for group activities and interactions\n"
            "- Energy derived from social situations\n"
            "- Quick responses and active participation\n"
            "- Focus on external activities and experiences"
        )
    }

    # Extract all characteristics
    detected_chars = extract_characteristics_from_text(text_content)

    if not detected_chars:
        return "No valid characteristics found in the text.", {}

    # Build the prompt
    prompt_parts = ["Choose the best result overall if one is clearly better. Otherwise, identify the best result in each aspect separately from these two answers.\n"]

    for category, value in detected_chars.items():
        if value in characteristics_guide:
            prompt_parts.append(f"\nFor {value} {category}:\n{characteristics_guide[value]}")

    prompt = "\n".join(prompt_parts)

    return prompt, detected_chars

def save_prompt_to_file(prompt, detected_chars):
    """
    Saves the generated prompt and detected characteristics to a text file.
    """
    filename = "prompt_generated.txt"

    with open(filename, 'w', encoding='utf-8') as file:
        # Write detected characteristics
        file.write("Detected Characteristics:\n")
        for category, value in detected_chars.items():
            file.write(f"{category}: {value}\n")

        # Write prompt
        file.write("\nGenerated Prompt:\n")
        file.write(prompt)

    return filename

def main():
    # Get results from ChatGPT and Claude
    result_chatgpt = input("Please enter the result from ChatGPT:\n")
    result_claude = input("\nPlease enter the result from Claude:\n")

    # Combine the results
    combined_text = f"{result_chatgpt}\n{result_claude}"

    # Generate prompt based on the content
    generated_prompt, detected_characteristics = generate_analysis_prompt(combined_text)

    # Save to file
    filename = save_prompt_to_file(generated_prompt, detected_characteristics)

    # Print results
    print("\nDetected Characteristics:")
    for category, value in detected_characteristics.items():
        print(f"{category}: {value}")
    print("\nGenerated Prompt:\n")
    print(generated_prompt)
    print(f"\nPrompt has been saved to: {filename}")

if __name__ == "__main__":
    main()

Please enter the result from ChatGPT:
**Here is a persuasive explanation of the movie *Inside Man* for a person with a 'Liking' dominant persuasion strategy, 'muscular' personality, 'high' uncertainty avoidance, and 'extrovert' tendencies:**  *"You’re going to love *Inside Man* — it’s one of those rare thrillers that instantly hooks you with a clever mastermind and a compelling cast you’ll genuinely admire. Denzel Washington and Clive Owen give strong, commanding performances that make it easy to root for them, even on opposite sides of the law. The story unfolds with precision — everything is calculated, and nothing is left to chance, which makes it feel incredibly satisfying and secure to follow. And honestly, this is the perfect movie to watch with friends — every twist invites conversation, debate, and shared excitement. You’ll enjoy the energy, the suspense, and the vibe of being in on something smart and powerful."*  ---  ### 💡 Breakdown by Personality Trait:  * **🧠 *Liking (Domi

In [5]:
def extract_characteristics_from_text(text):
    """
    Extracts all relevant characteristics from text.
    Returns a dictionary of detected characteristics.
    """
    text = text.lower()
    characteristics = {}

    # Persuasion Strategy
    if 'unity' in text:
        characteristics['persuasion'] = 'Unity'
    elif 'authority' in text:
        characteristics['persuasion'] = 'Authority'
    elif 'liking' in text:
        characteristics['persuasion'] = 'Liking'

    # Uncertainty Avoidance
    if 'high uncertainty' in text or 'risk averse' in text:
        characteristics['uncertainty'] = 'High'
    elif 'low uncertainty' in text or 'risk taking' in text:
        characteristics['uncertainty'] = 'Low'

    # Masculinity/Femininity
    if 'muscular' in text or 'masculinity' in text:
        characteristics['gender_dimension'] = 'Masculine'
    elif 'feminine' in text or 'femininity' in text:
        characteristics['gender_dimension'] = 'Feminine'

    # Personality Type
    if 'introvert' in text or 'introverted' in text:
        characteristics['personality'] = 'Introvert'
    elif 'extrovert' in text or 'extroverted' in text:
        characteristics['personality'] = 'Extrovert'

    return characteristics
def save_formatted_prompt(chatgpt_result, claude_result, prompt, detected_chars, characteristics_guide):
    """
    Saves the formatted prompt and analysis to a text file.
    """
    filename = "prompt_generated.txt"

    with open(filename, 'w', encoding='utf-8') as file:
        # Write the initial comparison instruction
        file.write("To make the persuasive explanation ,Choose the best result from these two answers , if one is clearly better. Otherwise, identify the best result in each aspect separately from these two answers and show the results in the output. Then rewrite the persuasive explanation based on the expalantions on each of the aspects written below and show what each part of your response corresponds to each aspect.Then translate you response to Persian.:\n\n")

        # Write ChatGPT and Claude results
        file.write("Answer1\n")
        file.write(chatgpt_result + "\n\n")
        file.write("Answer2:\n")
        file.write(claude_result + "\n\n")

        # Write characteristics explanation introduction
        if detected_chars:
            file.write("And here are some explanations about each of these aspects that you can consider while making the persuasive explanations:\n\n")

            # Write each characteristic's explanation
            for category, value in detected_chars.items():
                if value in characteristics_guide:
                    file.write(f"For {value} {category}:\n")
                    file.write(f"{characteristics_guide[value]}\n\n")

    # Read the file content and return it along with the filename
    with open(filename, 'r', encoding='utf-8') as file:
        file_content = file.read()

    return filename, file_content

def main():
    # Characteristics guide dictionary (same as before)
    characteristics_guide = {
        'Unity': (
            "Create an explanation that:\n"
            "- Emphasize that this movie is popular among people of a similar age or group, creating a sense of commonality and shared experience\n"
            "- Emphasizes shared experiences and common values\n"
            "- Highlights collective journey and mutual understanding\n"
            "- Shows how the story reflects universal human experiences\n"
            "- Creates a sense of belonging and community connection\n"
            "- Demonstrates how the protagonist's journey relates to common life experiences"
        ),
        'Authority': (
            "Create an explanation that:\n"
            "- Emphasizes expert opinions and critical acclaim\n"
            "- Highlights awards, recognition, and professional reviews\n"
            "- References influential figures who praised the film\n"
            "- Includes statistics and expert analysis\n"
            "- Demonstrates the film's impact on cinema and culture"
        ),
        'Liking': (
            "Create an explanation that:\n"
            "- Emphasizes emotional connections and relatable characters\n"
            "- Highlights personal and touching moments\n"
            "- Shows how the story creates emotional bonds\n"
            "- Focuses on likable aspects of characters\n"
            "- Demonstrates how viewers develop attachment to the story"
        ),
        'High': (
            "Consider these uncertainty avoidance aspects:\n"
            "- Preference for clear rules and structured situations\n"
            "- Stress the preference for clear rules and structured situations, highlighting the film's emphasis on predictability, planning, and stability\n"
            "- emphasize on unpredictibility of the screenplay\n"
            "- Focus on planning and risk mitigation\n"
            "- Emphasis on stability and predictability\n"
            "- Attention to detail and careful decision-making\n"
            "- Resistance to unconventional or ambiguous elements"
        ),
        'Low': (
            "Consider these uncertainty acceptance aspects:\n"
            "- Comfort with ambiguous or unstructured situations\n"
            "- Highlight the movie’s unpredictable twists and turns, emphasizing how it keeps viewers guessing and maintains suspense\n"
            "- Embrace of risk and innovation\n"
            "- Flexibility in approach and methodology\n"
            "- Openness to new experiences and changes\n"
            "- Tolerance for unexpected developments"
        ),
        'Masculine': (
            "Consider these masculine cultural elements:\n"
            "- Focus on achievement and success\n"
            "- Emphasis on competition and assertiveness\n"
            "- Value placed on material rewards\n"
            "- Clear goal-oriented actions\n"
            "- Prominence of ambitious pursuits"
        ),
        'Feminine': (
            "Consider these feminine cultural elements:\n"
            "- Focus on quality of life and relationships\n"
            "- Emphasis on cooperation and consensus\n"
            "- Value placed on caring and nurturing\n"
            "- Attention to work-life balance\n"
            "- Importance of social connections"
        ),
        'Introvert': (
            "Consider these introverted characteristics:\n"
            "- Internal processing and reflection\n"
            "- Emphasize the experience of watching the movie alone for a deeper, personal engagement with its themes\n"
            "- Should emphasize on watching the movie alone\n"
            "- Preference for deeper, one-on-one interactions\n"
            "- Need for personal space and solitude\n"
            "- Thoughtful and measured responses\n"
            "- Focus on internal growth and understanding"
        ),
        'Extrovert': (
            "Consider these extroverted characteristics:\n"
            "- External processing and social engagement\n"
            "- Emphasize the enjoyment of watching the movie with others, highlighting the social experience and opportunities for shared reactions\n"
            "- Should emphasize on watching the movie with other people\n"
            "- Preference for group activities and interactions\n"
            "- Energy derived from social situations\n"
            "- Quick responses and active participation\n"
            "- Focus on external activities and experiences"
        )
    }

    # Get results from ChatGPT and Claude
    result_chatgpt = input("Please enter the result from ChatGPT:\n")
    result_claude = input("\nPlease enter the result from Claude:\n")

    # Extract characteristics
    text = f"{result_chatgpt}\n{result_claude}"
    detected_chars = extract_characteristics_from_text(text)

    # Save formatted output and display content
    filename, file_content = save_formatted_prompt(
        result_chatgpt,
        result_claude,
        "",  # Empty string for prompt since we're reformatting
        detected_chars,
        characteristics_guide
    )

    print(f"\nFormatted output has been saved to: {filename}")
    print("\nContent of the generated text file:")
    print(file_content)
    print("\nDetected Characteristics:")
    for category, value in detected_chars.items():
        print(f"{category}: {value}")

if __name__ == "__main__":
    main()


Please enter the result from ChatGPT:
**Here is a persuasive explanation of the movie *Inside Man* for a person with a 'Liking' dominant persuasion strategy, 'muscular' personality, 'high' uncertainty avoidance, and 'extrovert' tendencies:**  *"You’re going to love *Inside Man* — it’s one of those rare thrillers that instantly hooks you with a clever mastermind and a compelling cast you’ll genuinely admire. Denzel Washington and Clive Owen give strong, commanding performances that make it easy to root for them, even on opposite sides of the law. The story unfolds with precision — everything is calculated, and nothing is left to chance, which makes it feel incredibly satisfying and secure to follow. And honestly, this is the perfect movie to watch with friends — every twist invites conversation, debate, and shared excitement. You’ll enjoy the energy, the suspense, and the vibe of being in on something smart and powerful."*  ---  ### 💡 Breakdown by Personality Trait:  * **🧠 *Liking (Domi